# Important!

In [36]:
# Following this tutorial: https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568
# TODO: This tutorial has enhancements: https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f

import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
import lxml

In [160]:
# read data
df = pd.read_csv('raw_data/fulltrain.csv', names=['labels', 'text'])
test_df = pd.read_csv('raw_data/balancedtest.csv', names=['labels', 'text'])
df.head()

,labels,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


In [161]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        return: modified initial string
    """
    text = str(text).replace(r'http[\w:/\.]+','') # removing urls
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwords from text
    return text

df['text'] = df['text'].apply(clean_text)
test_df['text'] = test_df['text'].apply(clean_text)

In [162]:
# create x and y
X = df['text']
y = df['labels']

# test x and y
X_test = test_df['text']
y_test = test_df['labels']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
my_tags = ['trusted', 'satire', 'hoax', 'propaganda']

# Naive Bayes (NB)

In [29]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

print('VALIDATION SET')
y_pred = nb.predict(X_val)

print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))


print('TEST SET')
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_tags))

VALIDATION SET
accuracy 0.6306884082690865
              precision    recall  f1-score   support

     trusted       0.85      0.65      0.74      4244
      satire       0.99      0.09      0.16      2065
        hoax       0.52      1.00      0.68      5313
  propaganda       1.00      0.34      0.50      3035

    accuracy                           0.63     14657
   macro avg       0.84      0.52      0.52     14657
weighted avg       0.78      0.63      0.59     14657

TEST SET
accuracy 0.362
              precision    recall  f1-score   support

     trusted       0.64      0.29      0.40       750
      satire       0.75      0.00      0.01       750
        hoax       0.29      1.00      0.46       750
  propaganda       1.00      0.16      0.27       750

    accuracy                           0.36      3000
   macro avg       0.67      0.36      0.28      3000
weighted avg       0.67      0.36      0.28      3000



# Support Vector Machine (SVM)

In [30]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

print('VALIDATION SET')
y_pred = sgd.predict(X_val)

print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred,target_names=my_tags))


print('TEST SET')
y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

VALIDATION SET
accuracy 0.8894043801596507
              precision    recall  f1-score   support

     trusted       0.82      0.95      0.88      4244
      satire       0.95      0.86      0.90      2065
        hoax       0.89      0.95      0.92      5313
  propaganda       0.97      0.71      0.82      3035

    accuracy                           0.89     14657
   macro avg       0.91      0.87      0.88     14657
weighted avg       0.90      0.89      0.89     14657

TEST SET
accuracy 0.6766666666666666
              precision    recall  f1-score   support

     trusted       0.70      0.77      0.73       750
      satire       0.72      0.35      0.47       750
        hoax       0.55      0.95      0.70       750
  propaganda       0.90      0.65      0.75       750

    accuracy                           0.68      3000
   macro avg       0.72      0.68      0.66      3000
weighted avg       0.72      0.68      0.66      3000



# Logistic Regression (LR)

In [140]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer()),
                   ('clf', LogisticRegression(random_state=0, max_iter=10000)),
                  ])
logreg.fit(X_train, y_train)

print('VALIDATION SET')
y_pred = logreg.predict(X_val)

print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))


print('TEST SET')
y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_tags))

VALIDATION SET
accuracy 0.9502626731254691
              precision    recall  f1-score   support

     trusted       0.93      0.97      0.95      4244
      satire       0.97      0.92      0.95      2065
        hoax       0.95      0.98      0.96      5313
  propaganda       0.97      0.90      0.93      3035

    accuracy                           0.95     14657
   macro avg       0.95      0.94      0.95     14657
weighted avg       0.95      0.95      0.95     14657

TEST SET
accuracy 0.7383333333333333
              precision    recall  f1-score   support

     trusted       0.81      0.80      0.81       750
      satire       0.82      0.41      0.54       750
        hoax       0.60      0.88      0.71       750
  propaganda       0.83      0.87      0.85       750

    accuracy                           0.74      3000
   macro avg       0.76      0.74      0.73      3000
weighted avg       0.76      0.74      0.73      3000



# Word2Vec + LR

In [113]:
# load word vectors 
from gensim.models import Word2Vec
wv = gensim.models.KeyedVectors.load_word2vec_format("./GoogleNews-vectors-negative300.bin", binary=True)
wv.init_sims(replace=True)

/var/folders/sx/07cm72r10wsb6gt_3x9rvgkm0000gn/T/ipykernel_4633/3448910852.py:4: DeprecationWarning: Call to deprecated `init_sims` (Use fill_norms() instead. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  wv.init_sims(replace=True)


In [126]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.key_to_index:
            mean.append(wv.vectors[wv.key_to_index[word]])
            all_words.add(wv.key_to_index[word])

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list])
    
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

train_tokenized = X_train.apply(lambda r: w2v_tokenize_text(r)).values
val_tokenized = X_val.apply(lambda r: w2v_tokenize_text(r)).values
test_tokenized = X_test.apply(lambda r: w2v_tokenize_text(r)).values

X_train_word_average = word_averaging_list(wv, train_tokenized)
X_val_word_average = word_averaging_list(wv, val_tokenized)
X_test_word_average = word_averaging_list(wv, test_tokenized)

In [130]:
logreg = LogisticRegression(random_state=0, max_iter=10000)
logreg = logreg.fit(X_train_word_average, y_train)

print('VALIDATION SET')
y_pred = logreg.predict(X_val_word_average)

print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))


print('TEST SET')
y_pred = logreg.predict(X_test_word_average)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_tags))

VALIDATION SET
accuracy 0.8795114962134134
              precision    recall  f1-score   support

     trusted       0.86      0.89      0.88      4244
      satire       0.88      0.81      0.85      2065
        hoax       0.89      0.94      0.91      5313
  propaganda       0.88      0.80      0.84      3035

    accuracy                           0.88     14657
   macro avg       0.88      0.86      0.87     14657
weighted avg       0.88      0.88      0.88     14657

TEST SET
accuracy 0.6783333333333333
              precision    recall  f1-score   support

     trusted       0.75      0.69      0.71       750
      satire       0.65      0.40      0.49       750
        hoax       0.57      0.86      0.68       750
  propaganda       0.81      0.77      0.79       750

    accuracy                           0.68      3000
   macro avg       0.69      0.68      0.67      3000
weighted avg       0.69      0.68      0.67      3000



Seems like it was pretty bad... in that case, we shall try...

# Doc2Vec + LR

In [133]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
import re

def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(TaggedDocument(v.split(), [label]))
    return labeled

X_train_labelled = label_sentences(X_train, 'Train')
X_val_labelled = label_sentences(X_val, 'Val')
all_data_train_val = X_train_labelled + X_val_labelled

X_test_labelled = label_sentences(X_test, 'Test')
all_data_train_test = X_train_labelled + X_test_labelled

In [135]:
model_dbow_train_val = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow_train_val.build_vocab([x for x in tqdm(all_data_train_val)])

for epoch in range(10):
    print(epoch)
    model_dbow_train_val.train(utils.shuffle([x for x in tqdm(all_data_train_val)]), total_examples=len(all_data_train_val), epochs=1)
    model_dbow_train_val.alpha -= 0.002
    model_dbow_train_val.min_alpha = model_dbow_train_val.alpha

model_dbow_train_test = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow_train_test.build_vocab([x for x in tqdm(all_data_train_test)])

for epoch in range(10):
    print(epoch)
    model_dbow_train_test.train(utils.shuffle([x for x in tqdm(all_data_train_test)]), total_examples=len(all_data_train_test), epochs=1)
    model_dbow_train_test.alpha -= 0.002
    model_dbow_train_test.min_alpha = model_dbow_train_test.alpha


100%|██████████| 48854/48854 [00:00<00:00, 923585.51it/s]


0


100%|██████████| 48854/48854 [00:00<00:00, 1932880.50it/s]


1


100%|██████████| 48854/48854 [00:00<00:00, 1734132.19it/s]


...

100%|██████████| 37197/37197 [00:00<00:00, 1593849.23it/s]


8


100%|██████████| 37197/37197 [00:00<00:00, 1682579.76it/s]


9


100%|██████████| 37197/37197 [00:00<00:00, 1551112.27it/s]


In [136]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors
    
train_vectors_dbow_with_val = get_vectors(model_dbow_train_val, len(X_train), 300, 'Train')
val_vectors_dbow = get_vectors(model_dbow_train_val, len(X_val), 300, 'Val')

train_vectors_dbow_with_test = get_vectors(model_dbow_train_test, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow_train_test, len(X_test), 300, 'Test')

/var/folders/sx/07cm72r10wsb6gt_3x9rvgkm0000gn/T/ipykernel_4633/3599969956.py:13: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  vectors[i] = model.docvecs[prefix]


In [139]:
logreg = LogisticRegression(n_jobs=1, C=1e5, random_state=0, max_iter=10000)
logreg = logreg.fit(train_vectors_dbow_with_val, y_train)

print('VALIDATION SET')
y_pred = logreg.predict(val_vectors_dbow)

print('accuracy %s' % accuracy_score(y_pred, y_val))
print(classification_report(y_val, y_pred, target_names=my_tags))

logreg = logreg.fit(train_vectors_dbow_with_test, y_train)

print('TEST SET')
y_pred = logreg.predict(test_vectors_dbow)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_tags))

VALIDATION SET
accuracy 0.9497850856246163
              precision    recall  f1-score   support

     trusted       0.95      0.95      0.95      4244
      satire       0.96      0.94      0.95      2065
        hoax       0.96      0.97      0.96      5313
  propaganda       0.94      0.92      0.93      3035

    accuracy                           0.95     14657
   macro avg       0.95      0.94      0.95     14657
weighted avg       0.95      0.95      0.95     14657

TEST SET
accuracy 0.6896666666666667
              precision    recall  f1-score   support

     trusted       0.82      0.77      0.79       750
      satire       0.66      0.35      0.46       750
        hoax       0.55      0.72      0.62       750
  propaganda       0.76      0.92      0.83       750

    accuracy                           0.69      3000
   macro avg       0.70      0.69      0.68      3000
weighted avg       0.70      0.69      0.68      3000



# BOW (with Keras)

In [165]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

X_train_bow = X_train
X_val_bow = X_val
X_test_bow = X_test

y_train_bow = y_train
y_val_bow = y_val
y_test_bow = y_test

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(X_train_bow) # only fit on train

X_train_bow = tokenize.texts_to_matrix(X_train_bow)
X_val_bow = tokenize.texts_to_matrix(X_val_bow)
X_test_bow = tokenize.texts_to_matrix(X_test_bow)

encoder = LabelEncoder()
encoder.fit(y_train_bow) # only fit on train
y_train_bow = encoder.transform(y_train_bow)
y_val_bow = encoder.transform(y_val_bow)
y_test_bow = encoder.transform(y_test_bow)

num_classes = np.max(y_train_bow) + 1
y_train_bow = utils.to_categorical(y_train_bow, num_classes)
y_val_bow = utils.to_categorical(y_val_bow, num_classes)
y_test_bow = utils.to_categorical(y_test_bow, num_classes)

batch_size = 32
epochs = 2

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(X_train_bow, y_train_bow,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

print('VALIDATION SET')
# y_pred = logreg.predict(val_vectors_dbow)

# print('accuracy %s' % accuracy_score(y_pred, y_val))
# print(classification_report(y_val, y_pred, target_names=my_tags))

score_val = model.evaluate(X_val_bow, y_val_bow,
                       batch_size=batch_size, verbose=1)
print('accuracy:', score_val[1])


print('TEST SET')
# y_pred = logreg.predict(test_vectors_dbow)

# print('accuracy %s' % accuracy_score(y_pred, y_test))
# print(classification_report(y_test, y_pred, target_names=my_tags))

score_test = model.evaluate(X_test_bow, y_test_bow,
                       batch_size=batch_size, verbose=1)
print('accuracy:', score_test[1])

Epoch 1/2
962/962 [==============================] - 6s 5ms/step - loss: 0.2864 - accuracy: 0.9034 - val_loss: 0.1556 - val_accuracy: 0.9447
Epoch 2/2
962/962 [==============================] - 5s 5ms/step - loss: 0.1398 - accuracy: 0.9532 - val_loss: 0.1382 - val_accuracy: 0.9494
VALIDATION SET
459/459 [==============================] - 1s 2ms/step - loss: 0.1599 - accuracy: 0.9429
accuracy: 0.9428941607475281
TEST SET
94/94 [==============================] - 0s 2ms/step - loss: 1.3537 - accuracy: 0.6783
accuracy: 0.6783333420753479
